In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50

In [6]:
# load data
df = pd.read_csv("Data_Marketing_Customer_Analysis_Round3.csv")
# create dataframes with numerical...
df_num = df[df.select_dtypes(include=[np.number]).columns].drop("total_claim_amount", axis=1)
# ...and categorical data only
df_cat = df[df.select_dtypes(include=[object]).columns]